In [1]:
from urllib.request import urlopen

from bs4 import BeautifulSoup

with urlopen("https://pokemon.fandom.com/wiki/List_of_Pok%C3%A9mon") as f:
    contents = f.read()


In [4]:
soup = BeautifulSoup(contents, "html.parser")


In [5]:
all_pokemon = []
for table in soup.find_all(class_="wikitable"):
    for row in table.find_all("tr"):
        pokemon = {}
        cells = row.find_all("td")
        contents = [s.text.strip() for s in cells]
        if contents:
            # print(f"num: {contents[0]}")
            pokemon["number"] = int(contents[0])
            # print(f"name: {contents[2]}")
            pokemon["name"] = contents[2]
            if cells[1].a:
                # print(f"image: {cells[1].a['href']}")
                pokemon["image"] = cells[1].a["href"]
            # else:
                # print(f"image: {cells[1]}")
            # print(f"type 1: {contents[3]}")
            pokemon["type 1"] = contents[3]
            if contents[4]:
                pokemon["type 2"] = contents[4]
                # print(f"type 2: {contents[4]}")
            # print()
            all_pokemon.append(pokemon)


In [6]:
len(all_pokemon)


1008

In [7]:
import json

with open("all_pokemon.json", "wt") as f:
    json.dump(all_pokemon, f, indent=4)


In [11]:
from reportlab.pdfgen import canvas
from reportlab.lib.units import inch
from reportlab.lib.pagesizes import LETTER


In [16]:
c = canvas.Canvas("pokedex.pdf", pagesize=LETTER)

vertical_margin = 0.5 * inch
horizontal_margin = 0.18 * inch
vertical_padding = 0.1 * inch
horizontal_padding = 0.1 * inch

POKEMON_PER_ROW = 6
POKEMON_PER_COLUMN = 5

pokemon_width = (
    LETTER[0] - horizontal_margin * 2 - (POKEMON_PER_ROW - 1) * horizontal_padding
) / (POKEMON_PER_ROW)

pokemon_height = (LETTER[1] - vertical_margin * 2 - (
    POKEMON_PER_COLUMN - 1
) * vertical_padding) / (POKEMON_PER_COLUMN) 

pokemon_per_page = POKEMON_PER_ROW * POKEMON_PER_COLUMN

new_page = True
for row_num, row_starting_index in enumerate(
    range(0, len(all_pokemon), POKEMON_PER_ROW)
):
    if new_page:
        c.translate(horizontal_margin, LETTER[1] - vertical_margin - pokemon_height)
        c.saveState()
        new_page = False
    row = all_pokemon[row_starting_index : row_starting_index + POKEMON_PER_ROW]
    c.saveState()
    for pokemon in row:
        c.saveState()
        c.rect(0, 0, pokemon_width, pokemon_height, stroke=1, fill=0)
        c.setFont("Helvetica", 10)
        c.drawCentredString(
            pokemon_width / 2, pokemon_height / 2 + 15, f"#{pokemon['number']}"
        )
        c.setFont("Helvetica", 12)
        c.drawCentredString(pokemon_width / 2, pokemon_height / 2, pokemon["name"])
        c.setFont("Helvetica", 10)
        c.translate(0, -20)
        c.drawCentredString(pokemon_width / 2, pokemon_height / 2, pokemon["type 1"])
        if "type 2" in pokemon:
            c.translate(0, -10)
            c.drawCentredString(
                pokemon_width / 2, pokemon_height / 2, pokemon["type 2"]
            )

        c.restoreState()
        c.translate(pokemon_width + horizontal_padding, 0)
    c.restoreState()
    c.translate(0, -pokemon_height - vertical_padding)
    if (row_num + 1) % POKEMON_PER_COLUMN == 0:
        c.restoreState()
        c.showPage()
        new_page = True
c.save()


In [15]:
!open pokedex.pdf